# Analysis of `vehicle-zone-tracking.xml`

In [334]:
import pprint, math, datetime, os, copy
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

# scenario = "berlin-1pct"
# scenario = "charlottenburg-no-meso-periodic"
# scenario = "normal-output"

scenarios = [
    "../simulation/performance/traci-n",
    "../simulation/performance/libsumo-n",
    "../simulation/performance/traci-meso-n",
    "../simulation/performance/libsumo-meso-n",
    "../simulation/performance/traci-rr",
    "../simulation/performance/libsumo-rr",
    "../simulation/performance/traci-meso-rr",
    "../simulation/performance/libsumo-meso-rr",
]

# scenario = "../simulation/performance/traci-n"
# scenario = "../simulation/performance/libsumo-n"
# scenario = "../simulation/performance/traci-meso-n"
# scenario = "../simulation/performance/libsumo-meso-n"
# scenario = "../simulation/performance/traci-rr"
# scenario = "../simulation/performance/libsumo-rr"
# scenario = "../simulation/performance/traci-meso-rr"
# scenario = "../simulation/performance/libsumo-meso-rr"


In [324]:
def get_totals(scenario):
    totals = {
        "trips": 0,
        "persons": 0,
        "distance": 0,
        "steps": 0,
        "sim_time": 0
    }

    trips_info_file = f"{scenario}/output/trips-info.xml"
    persons = []
    for event, element in etree.iterparse(trips_info_file, tag="tripinfo"):
        totals["trips"] += 1
        totals["distance"] += float(element.attrib["routeLength"])

        person, counter = element.attrib["id"].split("_")
        if person not in persons:
            totals["persons"] += 1
            persons.append(person)

    start_step = -1
    end_step = -1
    start_sim_time = -1
    end_sim_time = -1
    vehicle_summary_file = f"{scenario}/output/vehicle-summary.xml"
    for event, element in etree.iterparse(vehicle_summary_file, tag="step"):
        if start_step == -1:
            start_step = float(element.attrib["time"])
        else:
            end_step = float(element.attrib["time"])

        if start_sim_time == -1:
            start_sim_time = float(element.attrib["duration"])
        else:
            end_sim_time = float(element.attrib["duration"])

    totals["steps"] = end_step - start_step
    totals["sim_time"] = end_sim_time - start_sim_time

    return totals


In [325]:
# Distances in every zone for each trip
def get_distances_pt(scenario, freeze_zones=False):
    file_path = f"{scenario}/output/vehicle-zone-tracking.xml"
    distances_per_zone = {}
  
    for event, element in etree.iterparse(file_path, tag="timestep"):
        current_zone_timestep = element.attrib["zone-timestep"]

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            speed = float(vehicle.attrib["speed"])
            v_timestep = vehicle.attrib["zone-timestep"]

            distances = {}
            if vid in distances_per_zone:
                distances = distances_per_zone[vid]

            for polygon in vehicle.getchildren():
                pid_parts = polygon.attrib["id"].split("_")
                pid = pid_parts[0]
                p_timestep = pid_parts[1]

                if freeze_zones:
                    if p_timestep != v_timestep:
                        continue
                else:
                    if p_timestep != current_zone_timestep:
                        continue

                if pid.startswith("hole"):
                    hole, hole_counter, zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] -= speed
                    else:
                        distances[zone] = -speed
                else:
                    zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] += speed
                    else:
                        distances[zone] = speed
            
            distances_per_zone[vid] = distances

        element.clear()

    return distances_per_zone

distances_pt = get_distances_pt(scenarios[0])
pprint.pprint(distances_pt)

    '02': 241.084578911647,
                 '03': 142.7255265829159},
 '196086401_1': {'01': 545.922376700244},
 '196086401_4': {'01': 138.38598171920646},
 '198421001_2': {'01': 124.21143748082582,
                 '02': 116.51517339305147,
                 '03': 821.0951291620887},
 '198865301_2': {'01': 639.8939825736094},
 '199256001_3': {'01': 724.8288269028835,
                 '02': 187.82173839280406,
                 '03': 231.05817636552234},
 '199256001_6': {'01': 530.8102479351048,
                 '02': 189.11286455153444,
                 '03': 218.07331757962342},
 '201400001_3': {'01': 723.6769019638994,
                 '02': 409.51648628099315,
                 '03': 226.35801969640468},
 '201400001_4': {},
 '203869801_5': {'02': 244.16115692925197,
                 '03': 994.3753973826894,
                 '04': 719.2720168246899},
 '20728901_4': {'01': 669.250277546526,
                '02': 206.588349818424,
                '03': 218.79031893218584},
 '207941101_1

In [326]:
def get_distances_pp(distances_pt):
    distances_pp = {}

    for trip in distances_pt:
        person, counter = trip.split("_")
        trip_distances = distances_pt[trip]
        
        if person in distances_pp:
            person_distances = distances_pp[person]

            for zone in trip_distances:
                if zone in person_distances:
                    person_distances[zone] += trip_distances[zone]
                else:
                    person_distances[zone] = trip_distances[zone]
                    
            distances_pp[person] = person_distances

        else:
            distances_pp[person] = copy.deepcopy(trip_distances)
    
    return distances_pp

distances_pt = get_distances_pt(scenarios[0])
distances_pp = get_distances_pp(distances_pt)
# pprint.pprint(distances_pp)

x = list(distances_pp.keys())[25]
print(x)
pprint.pprint(distances_pp[x])
print()
for t in distances_pt:
    if t.startswith(x):
        pprint.pprint(distances_pt[t])

374008301
{'01': 1198.0638251381233, '02': 420.4360593016677, '03': 422.7787428534132}

{'01': 658.8977889230465, '02': 221.61239037770787, '03': 207.92935847804418}
{'01': 539.1660362150769, '02': 198.82366892395981, '03': 214.849384375369}


### Total trips per zone

In [327]:
def get_trips_pz(distances_pt):
    trips_pz = {}

    for trip in distances_pt:
        distances_pz = distances_pt[trip]
        for zone in distances_pz:
            if zone in trips_pz:
                trips_pz[zone] += 1
            else:
                trips_pz[zone] = 1
    
    return trips_pz

distances_pt = get_distances_pt(scenarios[0])
trips_pz = get_trips_pz(distances_pt)
pprint.pprint(trips_pz)

{'01': 239, '02': 153, '03': 138, '04': 2}


### Total persons per zone

In [328]:
def get_persons_pz(distances_pp):
    persons_pz = {}

    for person in distances_pp:
        distances = distances_pp[person]
        for zone in distances:
            if zone in persons_pz:
                if person not in persons_pz[zone]:
                    persons_pz[zone].append(person)
            else:
                persons_pz[zone] = [person]

    for zone in persons_pz:
        persons_pz[zone] = len(persons_pz[zone])

    return persons_pz

distances_pt = get_distances_pt(scenarios[0])
distances_pp = get_distances_pp(distances_pt)
persons_pz = get_persons_pz(distances_pp)
pprint.pprint(persons_pz)

{'01': 165, '02': 113, '03': 101, '04': 2}


In [329]:
def get_distances_pz(distances_pt):
    distances_pz = {}

    for trip in distances_pt:
        trip_distances = distances_pt[trip]
        for zone in trip_distances:
            if zone in distances_pz:
                distances_pz[zone] += trip_distances[zone] / 1000
            else:
                distances_pz[zone] = trip_distances[zone] / 1000

    return distances_pz

distances_pt = get_distances_pt(scenarios[0])
distances_pz = get_distances_pz(distances_pt)
pprint.pprint(distances_pz)

{'01': 135.1671107737726,
 '02': 38.9876011379387,
 '03': 37.82092669473563,
 '04': 0.926320776245935}


### Average distance per zone per trip

In [330]:
def get_avg_distances_pz_pt(distances_pt):
    avg_distances_pz_pt = {}

    for trip in distances_pt:
        distances_pz = distances_pt[trip]
        for zone in distances_pz:
            if zone in avg_distances_pz_pt:
                avg_distances_pz_pt[zone]["distance"] += distances_pz[zone]
                avg_distances_pz_pt[zone]["n"] += 1
            else:
                avg_distances_pz_pt[zone] = {
                    "distance": distances_pz[zone],
                    "n": 1
                }

    for zone in avg_distances_pz_pt:
        avg_distances_pz_pt[zone] = avg_distances_pz_pt[zone]["distance"] / avg_distances_pz_pt[zone]["n"]
    
    return avg_distances_pz_pt

distances_pt = get_distances_pt(scenarios[0])
avg_distances_pz_pt = get_avg_distances_pz_pt(distances_pt)
pprint.pprint(avg_distances_pz_pt)

{'01': 565.5527647438188,
 '02': 254.82092247018753,
 '03': 274.06468619373635,
 '04': 463.1603881229675}


### Average distance per zone per person

In [331]:
def get_avg_distances_pz_pp(distances_pp):

    avg_distances_pz_pp = {}

    for person in distances_pp:
        person_distances = distances_pp[person]
        for zone in person_distances:
            if zone in avg_distances_pz_pp:
                avg_distances_pz_pp[zone]["distance"] += person_distances[zone]
                avg_distances_pz_pp[zone]["n"] += 1
            else:
                avg_distances_pz_pp[zone] = {
                    "distance": person_distances[zone],
                    "n": 1
                }

    for zone in avg_distances_pz_pp:
        avg_distances_pz_pp[zone] = avg_distances_pz_pp[zone]["distance"] / avg_distances_pz_pp[zone]["n"]

    return avg_distances_pz_pp

distances_pt = get_distances_pt(scenarios[0])
distances_pp = get_distances_pp(distances_pt)
avg_distances_pz_pp = get_avg_distances_pz_pp(distances_pp)
pprint.pprint(avg_distances_pz_pp)

{'01': 819.1946107501367,
 '02': 345.0230189198114,
 '03': 374.46462073995673,
 '04': 463.1603881229675}


In [342]:
wb = Workbook()
ws = wb.active
write_header = True

header = []
row = []

def add(label, value):
    print(label)
    print(value)

    if write_header:
        header.append(label)

    row.append(value)

for s in scenarios[:1]:
    header = []
    row = []

    add("Scenario", os.path.basename(s))

    totals = get_totals(s)
    add("Total trips", totals["trips"])
    add("Total persons", totals["persons"])
    add("Total distance", totals["distance"])
    add("Total steps", totals["steps"])
    add("Total simulation time", f"{format(totals['sim_time'] / 1000, '.3f')} s")

    distances_pt = get_distances_pt(s)
    distances_pp = get_distances_pp(distances_pt)

    trips_pz = get_trips_pz(distances_pt)
    s = ""
    for zone in trips_pz:
        s += f"Zone {zone}: {trips_pz[zone]}\n"
    add("Total trips per zone", s)

    persons_pz = get_persons_pz(distances_pp)
    s = ""
    for zone in persons_pz:
        s += f"Zone {zone}: {persons_pz[zone]}\n"
    add("Total persons per zone", s)

    distances_pz = get_distances_pz(distances_pp)
    s = ""
    for zone in distances_pz:
        s += f"Zone {zone}: {format(distances_pz[zone], '.3f')} km\n"
    add("Total distances per zone", s)

    avg_distances_pz_pt = get_avg_distances_pz_pt(distances_pt)
    s = ""
    for zone in avg_distances_pz_pt:
        s += f"Zone {zone}: {format(avg_distances_pz_pt[zone], '.3f')} m\n"
    add("Average distance per zone per trip", s)

    avg_distances_pz_pp = get_avg_distances_pz_pp(distances_pp)
    s = ""
    for zone in avg_distances_pz_pp:
        s += f"Zone {zone}: {format(avg_distances_pz_pp[zone], '.3f')} m\n"
    add("Average distance per zone per person", s)


    if write_header:
        ws.append(header)
        write_header = False

    
    ws.append(row)

for row in ws.iter_rows():  
    for cell in row:      
        cell.alignment = Alignment(wrapText=True,vertical='top') 

for i, column_cells in enumerate(ws.columns, start=1):
    ws.column_dimensions[get_column_letter(i)].width = 40

for i, row in enumerate(ws.rows, start=1):
    ws.row_dimensions[i].height = 75

wb.save("sample.xlsx")


Scenario
traci-n
Total trips
1217
Total persons
584
Total distance
3740970.910000002
Total steps
86729.0
Total simulation time
249.606 s
Total trips per zone
Zone 01: 239
Zone 02: 153
Zone 03: 138
Zone 04: 2

Total persons per zone
Zone 01: 165
Zone 02: 113
Zone 03: 101
Zone 04: 2

Total distances per zone
Zone 01: 135.167 km
Zone 02: 38.988 km
Zone 03: 37.821 km
Zone 04: 0.926 km

Average distance per zone per trip
Zone 01: 565.553 m
Zone 02: 254.821 m
Zone 03: 274.065 m
Zone 04: 463.160 m

Average distance per zone per person
Zone 01: 819.195 m
Zone 02: 345.023 m
Zone 03: 374.465 m
Zone 04: 463.160 m

